In [1]:
from comet_ml import Experiment

import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import copy
import hdbscan
import astropy
import sys
import warnings 

from matplotlib import pyplot as plt
from matplotlib import cm

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from astropy.coordinates import Distance
from astropy import units as u
from astropy.cosmology import WMAP7
from astropy.io import fits

from collections import Counter
from pandas.api.types import is_numeric_dtype

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
# from xgboost import XGBRegressor

In [2]:
# insert at 1, 0 is the script path (or '' in REPL)
PATH = '/datadrive/azarodnyuk/GALAXY/'
# sys.path.insert(1, PATH)

In [3]:
# cd ../

In [4]:
y = pd.read_csv(PATH+'iGrID_rcsed_8_clean.csv', index_col='Unnamed: 0')
fltr = pd.notna(y.iGrID)

In [5]:
# Use the code below to download the original dataset

# hdul = fits.open('http://gal-03.sai.msu.ru/~vtoptun/photometry/rcsed_v2_8.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)

In [6]:
# hdul = fits.open(sys.path[1]+'rcsed_v2_8.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)

In [7]:
# cols = hdul[1].columns

In [8]:
# data = hdul[1].data

In [9]:
# hdul.close()
# del hdul

In [10]:
# DATA = pd.DataFrame(np.array(data).byteswap().newbyteorder())
# del data

In [11]:
DATA = pd.read_csv(PATH+'rcsed_v2_8.csv')

In [12]:
DATA.shape

(4109726, 480)

In [13]:
type_stats = {}
for c in DATA.columns:
    t = DATA[c].dtype
    if t in type_stats:
        type_stats[DATA[c].dtype] += 1
    else:
        type_stats[DATA[c].dtype] = 1

print(type_stats)

{dtype('int64'): 24, dtype('float64'): 447, dtype('O'): 9}


In [14]:
for c in DATA.columns:
    t = DATA[c].dtype
    if t == '<i8':
        print(c)

Unnamed: 0
ind
bestObjID_sdss
bossSpecObjID_sdss
mjd_sdss
plate_sdss
fiberID_sdss
q_z_2df
quality_6df
specid_6df
recno_uzc
obsid_lamost
lmjd_lamost
mjd_lamost
spid_lamost
fiberid_lamost
f_z_lega_c
f_spec_lega_c
OBJNO_deep2
OBJNO_deep3
Q_wigglez
NQ_gama
qual_2dFLenS
target_2dFLenS


In [15]:
DATA.drop(['ind', 'bestObjID_sdss'], inplace=True, axis=1)
DATA = DATA.select_dtypes(include='number')

In [16]:
DATA.shape

(4109726, 469)

In [17]:
type_stats = {}
for c in DATA.columns:
    t = DATA[c].dtype
    if t in type_stats:
        type_stats[DATA[c].dtype] += 1
    else:
        type_stats[DATA[c].dtype] = 1

print(type_stats)

{dtype('int64'): 22, dtype('float64'): 447}


In [19]:
len("percents of nan in filtered")

27

In [20]:
DATA.replace([-2147483648, -9223372036854775808,-32768,255, -999999488.0,
 99.0,
 -99.0,
 9999.0,
 -999.0,
 float('inf'),
 -9999.0], np.nan, inplace=True)

In [21]:
corr_matrix = DATA.corr()

In [34]:
corr_matrix[abs(corr_matrix.z_sdss)>0.4].z_sdss.sort_values(ascending=False)

z_sdss                  1.000000
z_lamost                0.975031
z_2dFLenS               0.935908
z_uzc                   0.881756
z_cfa                   0.808534
                          ...   
kcorr_aper3mag_Y_vhs   -0.416873
kcorr_aper2mag_Y_vhs   -0.416884
kcorr_petromag_Y_vhs   -0.416884
NQ_gama                -0.511431
qual_2dFLenS           -0.535428
Name: z_sdss, Length: 68, dtype: float64

In [35]:
col_name_corr = list(corr_matrix[abs(corr_matrix.z_sdss)>0.5].z_sdss.sort_values(ascending=False).index)

In [36]:
# col_name_corr

In [51]:
# DATA[col_name_corr].isna().sum()

In [37]:
# pip install keras

In [38]:
# pip install tensorflow

### Regression for redshift z

In [39]:
DATA.head()

,Unnamed: 0,ra,dec,z_sdss,zErr_sdss,specObjID_sdss,bossSpecObjID_sdss,mjd_sdss,plate_sdss,fiberID_sdss,...,kcorr_kronmag_z_panstarrs,kcorr_integmag_g_legacy,kcorr_integmag_r_legacy,kcorr_integmag_z_legacy,kcorr_aper2mag_g_legacy,kcorr_aper2mag_r_legacy,kcorr_aper3mag_g_legacy,kcorr_aper3mag_r_legacy,kcorr_aper6mag_g_legacy,kcorr_aper6mag_r_legacy
0,0.0,0.000019,-4.760800,2.466980,0.001469,7.919679e+18,2384066.0,56564.0,7034.0,360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.000091,24.902252,0.555913,0.000247,7.333218e+18,2133113.0,56543.0,6513.0,843.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,0.000346,-6.491940,0.514150,0.000124,8.047035e+18,2467493.0,56574.0,7147.0,831.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,0.000463,10.241074,0.654366,0.000156,6.960408e+18,1918240.0,56190.0,6182.0,346.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,0.000559,34.985602,0.146332,0.000021,8.044630e+18,2464948.0,56567.0,7145.0,274.0,...,0.012727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
Z = DATA['z_sdss']

In [39]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
x = imputer.fit_transform(DATA[col_name_corr])

In [40]:
x.shape

(4109726, 43)

In [41]:
# DATA.columns

In [42]:
size = x.shape[0]

full_sdss_indx = list(Z[~Z.isna()].index)

X = x[full_sdss_indx,:]
Y = Z[full_sdss_indx].to_numpy()

In [43]:
X.shape

(3154894, 43)

In [44]:
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)

train_X, val_X, train_y, val_y = train_test_split(X_scaled,Y, test_size = 0.3, random_state = 14)

In [45]:
# Create an experiment
# api_key could be found on https://www.comet.ml
experiment = Experiment(api_key='SoHcPReamhyjhD2j2S44j4KIJ',
                        project_name="galaxy", workspace='azarodnyuk')

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/azarodnyuk/galaxy/f5f6e50f2c6f4574a70c81c83f832385



In [46]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(int(train_X.shape[1]), kernel_initializer='normal',input_dim = train_X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(int(train_X.shape[1]/2), kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(int(train_X.shape[1]/4), kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(int(train_X.shape[1]/8), kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(int(train_X.shape[1]/16), kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error']) #'mean_absolute_error'
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 43)                1892      
_________________________________________________________________
dense_1 (Dense)              (None, 21)                924       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                220       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 3,097
Trainable params: 3,097
Non-trainable params: 0
_________________________________________________________________


In [47]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [48]:
NN_model.fit(train_X, train_y, epochs=15, batch_size=100, validation_split = 0.2, callbacks=callbacks_list,
             workers=20, use_multiprocessing=True)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/15
17653/17668 [============================>.] - ETA: 0s - loss: 0.0096 - mean_absolute_error: 0.0096
Epoch 00001: val_loss improved from inf to 0.00717, saving model to Weights-001--0.00717.hdf5
17668/17668 [==============================] - 52s 3ms/step - loss: 0.0096 - mean_absolute_error: 0.0096 - val_loss: 0.0072 - val_mean_absolute_error: 0.0072
Epoch 2/15
17664/17668 [============================>.] - ETA: 0s - loss: 0.0038 - mean_absolute_error: 0.0038
Epoch 00002: val_loss improved from 0.00717 to 0.00488, saving model to Weights-002--0.00488.hdf5
17668/17668 [==============================] - 53s 3ms/step - loss: 0.0038 - mean_absolute_error: 0.0038 - val_loss: 0.0049 - val_mean_absolute_error: 0.0049
Epoch 3/15
17658/17668 [============================>.] - ETA: 0s - loss: 0.0034 - mean_absolute_error: 0.0034
Epoch 00003: val_loss improved from 0.00488 to 0.00336, saving model to Weights-003--0.00336.hdf5
17668/17668 [==============================] - 54s 3ms/step -

In [49]:
predictions = NN_model.predict(val_X)

In [50]:
sklearn.metrics.r2_score(val_y, predictions)  #0.9998067818472489      0.6159740382704414

0.9998425041764135

### Reconstruct the redshift

In [53]:
nan_z_indx = list(Z[Z.isna()].index)

In [54]:
X_reconstruct = x[nan_z_indx,:]

In [55]:
z_reconstructed = NN_model.predict(X_reconstruct)

In [68]:
z_reconstructed.flatten()

array([3.0501912, 3.0645072, 3.0714037, ..., 2.49491  , 3.0487704,
       2.4972386], dtype=float32)

In [60]:
len(nan_z_indx)

954832

In [74]:
Z[nan_z_indx] = z_reconstructed.flatten()

In [75]:
Z.isna().sum()

0

In [82]:
Z.name = 'z_regres'

In [83]:
Z.to_csv('z_reconstructed.csv', index = False)

In [84]:
pd.read_csv('z_reconstructed.csv')

,z_regres
0,2.466980
1,0.555913
2,0.514150
3,0.654366
4,0.146332
...,...
4109721,3.049582
4109722,3.047023
4109723,2.494910
4109724,3.048770
